In [ ]:
import json
from pathlib import Path
import re
import pandas as pd
import plotly.express as px

In [ ]:
def load_scores(name):
    p = re.compile(r".*\\(.*)\\.*")
    data = {}
    for fname in Path("./evals/").glob(f"*/{name}"):
        with fname.open("rb") as fp:
            ckp_data = json.load(fp)
            ckp_data = dict((int(k), ckp_data[k]["test_score"]) for k in ckp_data.keys())
            data[p.match(str(fname)).groups()[0]] = ckp_data
    return data

def convert_to_df(data):
    a = []
    for k1, d in data.items():
        for k2, v in d.items():
            if k1.endswith("-bt-500k"):
                a.append({"model" : f"base+BT [{k1[:5]}]", "ckp": k2, "score": v})
            elif k1.endswith("+bt-250k"):
                a.append({"model" : f"base [{k1[:5]}]", "ckp": k2, "score": v})
            elif k1.endswith("-mixed-500k"):
                a.append({"model" : f"extended [{k1[:5]}]", "ckp": k2, "score": v})
    df = pd.DataFrame(a)
    return df

In [ ]:
px.line(
    convert_to_df(load_scores("flores_dev_results.json")), 
    x="ckp", y="score", color="model", range_x=(5000, 50000), range_y=(9, 15)
)

In [ ]:
convert_to_df(load_scores("wmt_results.json"))